<a href="https://colab.research.google.com/github/lucy483/Coding/blob/main/Workshops/workshop_01/03_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Application to molecular property data

<a rel="license" href="https://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="width=50" src="https://licensebuttons.net/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

**Authors**:
- Dr Antonia Mey (antonia.mey@ed.ac.uk)
- Ryan Zhu


**Questions:**
- How can we use regressions and clustering on real data?

**Objectives:**
- Load the properties dataset and explore what it contains
- Identify two properties that might correlate and learn a linear regression model

**Jupyter cheat sheet**:
- to run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- to get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;

The data cosists of calculated HOMO and LUMO of 561 small molecules, at b3lyp/def2tzvp level using density functional theory.
We are only working with computational data and we DO NOT compare to experiment. It is taken from [here](https://www.kaggle.com/datasets/aideesis/small-compounds-molecular-orbitals-dataset?resource=download).

## Google Colab installs

<div class="alert alert-warning">
The following cell installs necessary packages and downloads data if you are running this tutorial using Google Colab.<br>
<b><i>Run this cell only if you are using Google Colab!</i></b></div>

In [3]:
!if [ -n "$COLAB_RELEASE_TAG" ]; then git clone https://github.com/Edinburgh-Chemistry-Teaching/ATCP_ML-workshop; fi
import os
os.chdir(f"ATCP-ML-workshop{os.sep}Workshops{os.sep}workshop_01")

Cloning into 'ATCP_ML-workshop'...
fatal: could not read Username for 'https://github.com': No such device or address


FileNotFoundError: [Errno 2] No such file or directory: 'ATCP-ML-workshop/Workshops/workshop_01'

## 1. Loading the data

In [7]:
# Imports
import glob
import pandas as pd
import numpy as np
import kagglehub
from kagglehub import KaggleDatasetAdapter
import matplotlib.pyplot as plt
from sklearn import decomposition
from sklearn import cluster
import os
import warnings
warnings.filterwarnings("ignore")

In [9]:
filename = "molecular_descriptors.csv"
data = pd.read_csv(filename)
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'molecular_descriptors.csv'

## 2. Exploring the data

The data consists of names for compounds, as well as various properties. The best way to familiarise with data is by plotting it.


<div class="alert alert-success">
<b>Task 1:</b> Exploring the data

- Plot different properties of the data frame to see how correlated they are.
- Can you see any easy trends? Can you identify two properties that seem to be correlated to try and create a regression model?

In [ ]:
### Your solution here:


<details>
<summary> <mark> Solution: </mark> </summary>

```Python

features_start_at = list(data.columns).index("ho")
feature_names = data.columns[features_start_at:]

target_column = 'molecular_weight'

# Create scatter plots for each column against the target column
for column in feature_names:
    if column != target_column:
        plt.figure(figsize=(6, 4))
        plt.scatter(data[target_column], data[column])
        plt.xlabel(target_column)
        plt.ylabel(column)
        plt.title(f'{column} vs. {target_column}')
        plt.grid(True)
        plt.show()
```

</details>

## 3. Regression analysis


<div class="alert alert-success">
<b>Task 1: Pick two correlated properties and perform a regression analysis </b>

- How predictive is your regression on the test data based on different test/train splits?

</div>

In [ ]:
### Your solution here:




<details>
<summary> <mark> Regression Testing Solution </mark> </summary>

As an example, we pick molecular weight v. logP.
If you want to know more about the partition coefficient the [Wikipedia](https://en.wikipedia.org/wiki/Partition_coefficient) entry is a good starting point

Understanding logP is important for making drug molecules bioavailable in the body and an important property in drug design.

```Python

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# splitting data
indices = np.arange(len(data['logp']))
x = data[['molecular_weight']]
y = data[['logp']]
x_train, x_test, y_train, y_test, train_idx, test_idx = train_test_split(
   x ,y ,indices, test_size=0.33, random_state=42)


# Create linear regression object
regr = LinearRegression()

# Train the model using the training sets
regr.fit(x_train, y_train)

# Make predictions using the testing set and training set to work out the loss from the mean square error
y_pred_test = regr.predict(x_test)
y_pred_train = regr.predict(x_train)

# The mean squared error that will give you the loss
training_loss = mean_squared_error(y.values[train_idx], y_pred_train)
test_loss = mean_squared_error(y.values[test_idx], y_pred_test)

# The coefficient of determination: 1 is a perfect prediction and gives you an idea of the goodness of fit of the model
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred_test))

# The coefficients these are the coefficients you have successfully fit the model with
print("Coefficients: \n", regr.coef_)


# Plot outputs
figure = plt.figure(figsize=(6,6))
plt.plot(x.values[test_idx], y_test, "^", label="Testing labels truth", color='blue', alpha=0.3)
plt.plot(x.values[test_idx], y_pred_test, "+", label="Testing labels prediction", color='purple')
plt.text(0, 20, f"Training Loss: {training_loss:.5f}")
plt.text(0, 21, f"Testing Loss: {test_loss:.5f}")
plt.xlabel('molecular weight')
plt.ylabel('logP')
plt.legend()

```

</details>

## 4. Clustering analysis on the dataset

<div class="alert alert-success">
<b>Task 3: Clustering analysis of molecular descriptors </b>

- Perform a k-means clustering on fermi energies v. molecular weight
- Perform a DBSCAN clustering on fermi energies v. molecular weight
- Does it make sense to cluster this data? Can you draw any conclusions from the clustering?

</div>

In [ ]:
### Your solution here


<details>
<summary> <mark> Solution for k-means:</mark> </summary>

```Python

import sklearn.cluster as skl_cluster
import numpy as np

new_data = data[['molecular_weight','fermi']]

# kmeans
Kmean = skl_cluster.KMeans(n_clusters=3)
Kmean.fit(new_data)
clusters = Kmean.predict(new_data)

plt.scatter(new_data.values[:, 0], new_data.values[:, 1], s=5, linewidth=0, c=clusters)
for cluster_x, cluster_y in Kmean.cluster_centers_:
    plt.scatter(cluster_x, cluster_y, s=100, c='orange', marker='+')
plt.show()

```

</details>

<details>
<summary> <mark> Solution for DBSCAN</mark> </summary>

```Python

import sklearn.cluster as skl_cluster
import numpy as np

new_data = data[['molecular_weight','fermi']]
new_data = data[['molecular_weight','fermi']]

# Try different values of eps and min_samples
db = skl_cluster.DBSCAN(eps=5, min_samples=3)
db.fit(new_data)
clusters = db.labels_.astype(int)
no_clusters = len(np.unique(clusters) )
no_noise = np.sum(np.array(clusters) == -1, axis=0)

print(f'Estimated no. of clusters: {no_clusters}')
print(f'Estimated no. of noise points: {no_noise}')

plt.scatter(new_data.values[:,0], new_data.values[:,1], c=clusters, s=8, lw=0, marker="o")
plt.show()

```

</details>

## END

---